This notebook contains rough working and thoughts for the resulting blog post. For the full, more polished post, visit [my website](https://barnabywreford.co.uk/problems/vector_orthogonality_and_llm_superposition/)

## Working and Rough Talking Points

Illustration of how the distance between points grows in higher dimensions by looking at all points within 0.5 distance of the centre point:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from itertools import combinations, product

def plot_line(ax, r):
    # 1D Line [0,1]
    ax.plot([0, 1], [0, 0], color='black')
    center = 0.5
    ax.plot(center, 0, 'ro')

    # Highlighted segment within radius r
    left = max(center - r, 0)
    right = min(center + r, 1)
    ax.plot([left, right], [0, 0], color='red', linewidth=5, alpha=0.3)

    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(-0.5, 0.5)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('1D Line')

def plot_square(ax, r):
    # 2D Square [0,1]^2
    square = plt.Rectangle((0, 0), 1, 1, fill=None, edgecolor='black')
    ax.add_patch(square)
    point = (0.5, 0.5)
    ax.plot(*point, 'ro')
    circle = plt.Circle(point, r, color='red', alpha=0.3)
    ax.add_patch(circle)
    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(-0.1, 1.1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('2D Square')

def plot_cube(ax, r):
    # 3D Cube [0,1]^3
    cube_edges = combinations(np.array(list(product([0, 1], repeat=3))), 2)
    for s, e in cube_edges:
        if np.sum(np.abs(s - e)) == 1:
            ax.plot3D(*zip(s, e), color="black")

    point = np.array([0.5, 0.5, 0.5])
    ax.scatter(*point, color='red')

    # Sphere around point
    u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:20j]
    x = r * np.cos(u) * np.sin(v) + point[0]
    y = r * np.sin(u) * np.sin(v) + point[1]
    z = r * np.cos(v) + point[2]
    ax.plot_surface(x, y, z, color='red', alpha=0.3)

    ax.set_xlim([-0.1, 1.1])
    ax.set_ylim([-0.1, 1.1])
    ax.set_zlim([-0.1, 1.1])
    ax.view_init(30, 30)
    ax.axis('off')
    ax.set_title('3D Cube')

def plot_all(r=0.2):
    fig = plt.figure(figsize=(12, 4))

    ax1 = fig.add_subplot(1, 3, 1)
    plot_line(ax1, r)

    ax2 = fig.add_subplot(1, 3, 2)
    plot_square(ax2, r)

    ax3 = fig.add_subplot(1, 3, 3, projection='3d')
    plot_cube(ax3, r)

    plt.tight_layout()
    plt.show()

# Example usage
plot_all(r=0.5)

It may seem relatively unintuitive as to why increasing the number of dimensions means random vectors become increasingly likely to be near-orthoganol.

One way to think about it is by looking at the mathematical expression for the angle between two vectors

In [ ]:
def find_angle(v1, v2):
    cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    radians = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # ensure numerical stability
    return np.degrees(radians)

When the two vectors are nearly orthagonal, this means $\cos{\theta}$ is nearly 0. If we assume that as we vary our vectors over different dimensions we keep the magnitude of those vectors the same, this must mean we are looking to our dot product $a \cdot b$ to achieve this near 0 behaviour. If we were to choose our vector directions randomly, then this dot product is a summation of random numbers.

To more intuitively imagine a summation of random numbers, imagine rolling a dice with values \[-3,-2,1,1,2,3\], rolling it many times and taking the sum. If you roll it twice, the minimum and maximum values you can get is is \[-6,6\], and you are reasonably likely to get close to either extreme. However, if you roll the dice 100 times, the min and max values you can get is \[-300, 300\], however you are likely to get a score close to 0 as your rolls cancel out. The variance proportionally reducing as the number of trials increases is a fundamental of statistics.

Let's visualise this.

In [ ]:
def plot_histogram(data, max_bins=19, title=None, symmetric=True, center=0):
    data = np.asarray(data)
    unique_values = np.unique(data)
    num_unique = len(unique_values)

    if num_unique <= max_bins:
        # Use unique-value binning with small padding
        bins = np.concatenate([
            [unique_values[0] - 0.0001],
            (unique_values[:-1] + unique_values[1:]) / 2,
            [unique_values[-1] + 0.0001]
        ])
    else:
        if symmetric:
            # Symmetric binning around specified center
            max_offset = np.max(np.abs(data - center))
            bin_min = center - max_offset
            bin_max = center + max_offset
            bins = np.linspace(bin_min, bin_max, max_bins + 1)
        else:
            bins = np.linspace(np.min(data), np.max(data), max_bins + 1)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.hist(data, bins=bins, density=True, edgecolor='black')
    plt.title(title or f'Probability Distribution (n={len(data)})')
    plt.xlabel('Value')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.show()

In [ ]:
def simulate_dice_sums(n, trials=100_000):
    dice = np.array([-3, -2, -1, 1, 2, 3])
    results = np.random.choice(dice, size=(trials, n))
    sums = np.sum(results, axis=1)
    plot_histogram(sums, title=f'Dice Sums n={n}')

simulate_dice_sums(n=2)
simulate_dice_sums(n=100)

In our original equation, we divide by the magnitude of $a$ and $b$, in our dice example this is analogous to dividing by number of rolls. Therefore, the full analogy is to roll our dice n times, take the sum, then divide by n. I hope it is intuitive that as the number of rolls increases, we are likely to get a final score that is closer to 0.

However, let's visualise this to confirm the prediction.

In [ ]:
def simulate_weighted_dice_sums(n, trials=100_000):
    dice = np.array([-3, -2, -1, 1, 2, 3])
    results = np.random.choice(dice, size=(trials, n))
    sums = np.sum(results, axis=1) / (3*n)
    plot_histogram(sums, title=f'Weighted Dice Sums n={n}')

simulate_weighted_dice_sums(n=2)
simulate_weighted_dice_sums(n=100)

In the same way as with the dice, when calculating the angle between our vectors, the number of dimensions increases we are more likely to get near-orthogonal vectors.

Let's re-run the experiment, this time using random vectors instead of random dice rolls.

In [ ]:
def generate_angles(trials, d, memory_MB=1000):
    # randn returns a 64 bit float
    # so memory used = 2 * batch_size * d * 8
    batch_size = memory_MB*1024*1024 // (16*d)
    
    angles = np.empty(trials)
    for start in range(0, trials, batch_size):
        end = min(start + batch_size, trials)

        # Generate a batch of random vectors
        v1 = np.random.randn(end - start, d)
        v2 = np.random.randn(end - start, d)

        # Vectorized computation within batch
        dot_products = np.einsum('ij,ij->i', v1, v2)
        norms_v1 = np.linalg.norm(v1, axis=1)
        norms_v2 = np.linalg.norm(v2, axis=1)
        cos_angles = dot_products / (norms_v1 * norms_v2)
        angles[start:end] = np.degrees(np.arccos(np.clip(cos_angles, -1.0, 1.0)))
    
    return angles

def simulate_vector_angles(n, trials=100_000):
    angles = generate_angles(trials, n)
    plot_histogram(angles, center=90, title=f'Angle between 2 random vectors (dimensions={n})')

simulate_vector_angles(n=3)
simulate_vector_angles(n=100)

The number of near-orthagonal vectors in high dimensional spaces is often used as an explanation to why superposition appears in neural networks. 

The number of embedding dimensions for modern LLMs tends to increase as the models overall size is scaled. For example, for LLaMA 3, the number of dimensions is:
- 8B: 4096
- 70B: 8192
- 405B: 16,384

Let's take a look at the distribution of angles between any two random vectors in the embedding space of LLaMa 3's largest model at 405 Billion Parameters. 

In [ ]:
# might take a minute to compute
simulate_vector_angles(n=16384)

As a bonus, we can go further. As pairs of vectors become increasingly more orthoganol as the number of dimensions increases, so do larger groups of vectors.

Let's look at the minimum angle between any two vectors in a group, for different size groups in different dimensions.

In [ ]:
def simulate_multiple_vector_angles(dimensions, group_size, trials=100_000):
    angles = np.empty(trials)
    for i in range(trials):
        vectors = []
        for _ in range(group_size):
            vectors.append(np.random.randn(dimensions))

        min_angle = 180
        for j in range(group_size):
            for k in range(j+1, group_size):
                min_angle = min(min_angle, find_angle(vectors[j], vectors[k]))
                           
        angles[i] = min_angle
    
    plot_histogram(angles, symmetric=False, title=f'Group Vector Minimum Angle. (dimensions={dimensions}, group_size={group_size})')

simulate_multiple_vector_angles(100, 2)
simulate_multiple_vector_angles(100, 5)
simulate_multiple_vector_angles(1000, 5)

Remember these experiments are just to give an intuition about the sparsity of vectors by looking at random vectors. When LLMs are trained, they can essentially 'choose' which vectors to use (associate with a given feature), meaning they can choose a packing which maximises orthagonality between many vectors. 

You have seen that the average group of vectors becomes more orthogonal to each other as the number of dimensions grows, however also keep in mind that the 'best' group, that which has the maximal orthogonality between each vector, also grows. In fact, it grows much faster than these graphs might indicate - it is known that the maximum number of near-orthogonal vectors scales exponentially with the number of dimensions.

The Kabatiansky-Levenshtein Bound gives an asymptotic formula for approximating this value (technically referred to as the maximal size of a spherical code in n dimensions). Note the following KL values are just to illustrate the approximate shape and magnitude of the function, since the KL bound can become innaccurate for theta values close to 90 degrees.

In [ ]:
# Circle: Points no closer than 45 degrees
def plot_circle_with_dots():
    fig, ax = plt.subplots()
    circle = plt.Circle((0, 0), 1, color='lightgray', fill=False)
    ax.add_artist(circle)
    
    angles_deg = np.arange(0, 360, 45)  # Every 45 degrees
    angles_rad = np.deg2rad(angles_deg)
    
    x = np.cos(angles_rad)
    y = np.sin(angles_rad)
    
    ax.plot(x, y, 'o', color='red', label='45°-spaced dots')
    ax.set_aspect('equal')
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.set_title("Circle: Dots at ≥45°")
    ax.legend()
    plt.grid(True)
    plt.show()

# Sphere: Greedy algorithm to place points with angular separation ≥ 45°
def angular_distance(p1, p2):
    dot = np.dot(p1, p2)
    return np.arccos(np.clip(dot, -1.0, 1.0))

def generate_sphere_points(min_angle_deg):
    min_angle_rad = np.deg2rad(min_angle_deg)
    points = []

    # Generate candidates over the sphere using spherical coordinates
    phi_vals = np.linspace(0, np.pi, 100)
    theta_vals = np.linspace(0, 2*np.pi, 200)
    
    for phi in phi_vals:
        for theta in theta_vals:
            x = np.sin(phi) * np.cos(theta)
            y = np.sin(phi) * np.sin(theta)
            z = np.cos(phi)
            new_point = np.array([x, y, z])

            # Check if this point is sufficiently far from all existing points
            if all(angular_distance(new_point, p) >= min_angle_rad for p in points):
                points.append(new_point)

    return np.array(points)

def plot_sphere_with_dots():
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Draw a sphere
    u, v = np.mgrid[0:2*np.pi:100j, 0:np.pi:100j]
    xs = np.cos(u) * np.sin(v)
    ys = np.sin(u) * np.sin(v)
    zs = np.cos(v)
    ax.plot_surface(xs, ys, zs, color='lightblue', alpha=0.2)

    # Generate and plot dots
    sphere_points = generate_sphere_points(45)
    ax.scatter(sphere_points[:, 0], sphere_points[:, 1], sphere_points[:, 2], color='red')

    ax.set_box_aspect([1,1,1])
    ax.set_title(f"Sphere: Dots at ≥45° (Total: {len(sphere_points)})")
    plt.show()

# Run both plots
plot_circle_with_dots()
plot_sphere_with_dots()


In [ ]:
# calculate the exponent (base 10) of the Kabatyanskii-Levenshtein bound
def kl_bound(n, theta):
    if theta <= 0 or theta >= np.pi / 2:
        raise ValueError("Theta should be in the range (0, pi/2)")

    sin_theta = np.sin(theta)

    # apply KL formula
    term1 = (1 + sin_theta) / (2 * sin_theta)
    term2 = (1 - sin_theta) / (2 * sin_theta)
    log_term1 = np.log2(term1)
    log_term2 = np.log2(term2)

    kl_bound_value = (term1 * log_term1 - term2 * log_term2)

    return round((n * kl_bound_value) / np.log2(10), 2)

trial_values = ((16384, 88), (16384, 85), (16384, 80))
for n, theta in trial_values:
    bound = kl_bound(n, np.radians(theta))
    print(f"The maximum number of vectors with minimum angle separation {theta} in {n} dimensions is approx. 10^{bound}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_perpendicular_vectors(ax):
    labels = ["size", "red"]
    directions = [(1, 0), (0, 1)]

    for (dx, dy), label in zip(directions, labels):
        ax.quiver(0, 0, dx, dy, angles='xy', scale_units='xy', scale=1, color='red')
        if label == "size":
            ax.text(dx * 1.3, dy * 1.05, label, color='black', fontsize=16, fontweight='bold', ha='center', va='center')
        else:
            ax.text(dx * 1.1, dy * 1.1, label, color='black', fontsize=16, fontweight='bold', ha='center', va='center')

    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title("Two Perpendicular Vectors", fontsize=14, fontweight='bold')
    ax.grid(True)

def plot_pentagon_vectors(ax):
    labels = ["size", "red", "vehicle", "beauty", "animal"]
    n = len(labels)
    angle_step = 2 * np.pi / n
    radius = 1

    for i, label in enumerate(labels):
        angle = i * angle_step
        dx = radius * np.cos(angle)
        dy = radius * np.sin(angle)
        ax.quiver(0, 0, dx, dy, angles='xy', scale_units='xy', scale=1, color='red')
        ax.text(dx * 1.15, dy * 1.15, label, color='black', fontsize=16, fontweight='bold', ha='center', va='center')

    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title("Five Vectors in a Pentagon Shape", fontsize=14, fontweight='bold')
    ax.grid(True)

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
plot_perpendicular_vectors(axes[0])
plot_pentagon_vectors(axes[1])
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

def get_pentagon_vectors():
    labels = ["size", "red", "vehicle", "beauty", "animal"]
    n = len(labels)
    angle_step = 2 * np.pi / n
    radius = 1
    vectors = []
    for i in range(n):
        angle = i * angle_step
        dx = radius * np.cos(angle)
        dy = radius * np.sin(angle)
        vectors.append((np.array([dx, dy]), labels[i]))
    return vectors

def find_valid_combinations(vectors, p):
    valid = []
    for (i, (v1, l1)), (j, (v2, l2)) in combinations(enumerate(vectors), 2):
        M = np.column_stack((v1, v2))
        try:
            a, b = np.linalg.solve(M, p)
            if 0 <= a <= 1 and 0 <= b <= 1:
                valid.append((i, j, a, b))
        except np.linalg.LinAlgError:
            continue
    return valid

def plot_combination(vectors, i, j, a, b, p):
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_aspect('equal')
    ax.grid(True, linestyle='--', alpha=0.3)

    # Plot all basis vectors, label all
    for idx, (v, label) in enumerate(vectors):
        color = 'red' if idx in [i, j] else 'lightgray'
        ax.quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1, color=color, width=0.005)
        ax.text(v[0] * 1.15, v[1] * 1.15, label,
                color='black', fontsize=16, fontweight='bold',
                ha='center', va='center')

    # Draw the composition path: a * vi then b * vj
    vi = vectors[i][0]
    vj = vectors[j][0]
    p1 = a * vi
    ax.quiver(0, 0, p1[0], p1[1], angles='xy', scale_units='xy', scale=1, color='black')
    ax.quiver(p1[0], p1[1], p[0] - p1[0], p[1] - p1[1], angles='xy', scale_units='xy', scale=1, color='black')

    # Final point
    ax.plot(p[0], p[1], 'ro', markersize=8)
    ax.set_title(f"Combination: {vectors[i][1]} + {vectors[j][1]}", fontsize=14)
    
    lim = np.max(np.abs([v[0] for v in vectors] + [p])) * 1.5
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    plt.tight_layout()
    plt.show()

def visualize_vector_combinations(target_point):
    vectors = get_pentagon_vectors()
    valid_pairs = find_valid_combinations(vectors, target_point)

    if not valid_pairs:
        print("No valid vector pairs found where a, b ∈ [0, 1].")
        return

    for i, j, a, b in valid_pairs:
        plot_combination(vectors, i, j, a, b, target_point)

target = np.array([0.4, 0.3])
visualize_vector_combinations(target)

As can be observed from the group size graphs, if you increase the group size the average minimum angle between vectors decreases. Therefore, if you want to limit the probability of getting a certain level of similarity between vectors, there is a maximum limit to the group size you can use. In a similar vein, a crucial aspect of the KL bound formula is that the less orthogonality you have between your vectors - in other words the more similar your vectors are - the greater number of vectors you can pack into your space.

In deep learning, you want to be able to distinguish different feature vectors, so there is a limit to the allowable similarity between these vectors. If one imagines this allowable similarity as a limited resource, there are then two forces competing for it: a desire to increase the 'group size' (number of simultaneously active features) and a desire to increase the total number of vectors packed into the space (number of total features).

This is why the sparsity of features is a useful predictor for the amount of superposition in a neural network. 'sparsity of features' essentially means how often multiple features are active simultaneously as a proportion of the total features (similar to the group size in the experiment). Language can represent a vast array of ideas, but only a very small portion of those are present in a given sentence (or context window). This means the balance between those two forces tips towards the total number of features, and as discussed before in order to be able to increase total features by packing more vectors into the space then you must accept more similar vectors for those features. Greater similarity between feature vectors manifests as superposition.